In [5]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("jerry128/Musique-Ans-Train-CL-Sorted-By-Hops", cache_dir="./Musique", split="train")

ds

Dataset({
    features: ['id', 'paragraphs', 'question', 'answer', 'answer_aliases', 'context', 'citations'],
    num_rows: 19938
})

In [7]:
print(ds[0]["answer"])
print(ds[0]["answer_aliases"])

1960
[]


In [ ]:
# 将数据展平，处理成想要的格式，方便后续嵌入
from datasets import Dataset
flatten_ds = []
for id, paragraphs in zip(ds["id"], ds["paragraphs"]):
    for para in paragraphs:
        idx = para["idx"]  # 段落的索引
        text = para["paragraph_text"]  # 段落内容
        title = para["title"]  # 段落标题
        flatten_ds.append(
            {
                "id": f"{id}_{idx}",  # 创建一个唯一的段落ID
                "title": title,  # 段落标题
                "text": text,
                "hop": int(id[0]),
            }
        )

flatten_ds = Dataset.from_list(flatten_ds)
flatten_ds

18123


In [ ]:
# 可选是否保存
flatten_ds.save_to_disk("./Musique_flattened")
flatten_ds.to_parquet("./Musique_flattened/Musique_flattened.parquet")

In [2]:
def make_prefix(dp):
    question = dp["question"]

    # NOTE: also need to change reward_score/countdown.py
    prefix = f"""You are an efficient research assistant. Your goal is to answer the question by using tool to find information. You can search and scrape as many times as your want.
    If you need to use tool, you should first search for relevant documents, and then scrape the content of the documents to answer the question.
    If you find no further external knowledge needed, you can directly provide the answer inside <answer> and </answer>, without detailed illustrations. For example, <answer> Beijing </answer>.

**Available tool:**
1. `websearch`: Use a query to search for documents. It returns a list of results containing a `doc_id`, `title`, and `preview`.
2. `scrape`: Use a `doc_id` obtained from `websearch` to read the full content of a document.

**Instructions:**
- You MUST call tool using this exact format: <tool_call>{{"name": "tool_call_name", "args": {{"key": "value"}}}}</tool_call>
- Once you have gathered enough information, provide the final, concise answer inside <answer></answer> tags.

**Example:**
user:
When was the institute that owned The Collegian founded?

assistant:
<tool_call>{{"name": "websearch", "args": {{"query": "institute that owns The Collegian"}}}}</tool_call>

tool:
<result>[{{"id": "doc_101", "title": "The Collegian (Houston Baptist University)", "preview": "The Collegian is the official student publication of Houston Baptist University in Houston, Texas..."}},
 {{"id": "doc_202", "title": "Houston", "preview": "Several private institutions of higher learning are located within the city, including Houston Baptist University..."}}]</result>

assistant:
<tool_call>{{"name": "scrape", "args": {{"doc_ids": "doc_202"}}}}</tool_call>

tool:
<result>Several private institutions of higher learning—ranging from liberal arts colleges, such as The University of St. Thomas, to Rice University—are located within the city... Houston Baptist University, affiliated with the Baptist General Convention of Texas, offers bachelor's and graduate degrees. It was founded in 1960 and is located in the Sharpstown area in Southwest Houston.</result>

assistant:
<answer> 1960 </answer>

---
Now, answer the following question.
Question: {question}"""

    return prefix


# add a row to each data item that represents a unique id
def make_map_fn(split):

    def process_fn(example, idx):
        example["question"] = example["question"].strip()
        if example["question"][-1] != "?":
            example["question"] += "?"
        question = make_prefix(example)
        solution = {
            "target": example["answer"],
        }

        data = {
            "data_source": "oringinal_dataset(Musique)",
            "prompt": [
                {
                    "role": "user",
                    "content": question,
                }
            ],
            "ability": "fact-reasoning",
            "reward_model": {"style": "rule", "ground_truth": solution},
            "extra_info": {
                "split": split,
                "index": idx,
            },
        }
        return data

    return process_fn


In [1]:
train_dataset = ds.map(function=make_map_fn("train"), with_indices=True)

train_dataset.to_parquet("./Musique_RL/train.parquet")

NameError: name 'ds' is not defined

In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("jerry128/Musique-Ans-Eval-1000", cache_dir="./Musique", split="train")

train_dataset = ds.map(function=make_map_fn("test"), with_indices=True)

train_dataset.to_parquet("./Musique_RL/test.parquet")


README.md:   0%|          | 0.00/292 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

23425819